In [7]:
import pandas as pd
import os
import gzip
from tqdm.autonotebook import tqdm

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
max_sentiment = 20000
def process_df(df, aspect):
    df = df.drop(['reviewerID', 'asin', 'reviewerName', 'unixReviewTime', 'reviewTime'], axis=1)
        
    # add sentiment col
    df['sentiment'] = df.apply(process_sentiment, axis=1)
    
    # balance dataset 
    df = balance_dataset(df)
    
    # add aspect col
    df['aspect'] = df.apply(lambda r: aspect, axis=1)
    return df
    
def process_sentiment(row):
    if row['overall'] >= 4.0:
        return 'positive'
    if row['overall'] <= 2.0:
        return 'negative'
    return 'neutral'


def balance_dataset(df):
    pos_series = df['sentiment']=='positive'
    neu_series = df['sentiment']=='neutral'
    neg_series = df['sentiment']=='negative'
    
    pos_diff = df[pos_series]['sentiment'].count() - max_sentiment
    neu_diff = df[neu_series]['sentiment'].count() - max_sentiment
    neg_diff = df[neg_series]['sentiment'].count() - max_sentiment
    
    diffs = [pos_diff, neu_diff, neg_diff]
    print(diffs)

    
    # if the sum of all divs is negative, we can not balance the dataset
    if sum(diffs) < 0:
        print('No balancing possible')
        return df
    
    pos_sel = None
    neg_sel = None
    neu_sel = None
    
    remaining = pd.DataFrame()
    
    # pos has enough 
    if pos_diff >= 0:
        pos_sel = df[pos_series][:max_sentiment]
        remaining = remaining.append(df[pos_series][max_sentiment:], ignore_index=True)
        print(f'Pos enough: Remaining Count: {remaining["overall"].count()}')
    
    if neu_diff >= 0:
        neu_sel = df[neu_series][:max_sentiment]
        remaining = remaining.append(df[neu_series][max_sentiment:], ignore_index=True)
        print(f'Neutral enough: Remaining Count: {remaining["overall"].count()}')

        
    if neg_diff >= 0:
        neg_sel = df[neg_series][:max_sentiment]
        remaining = remaining.append(df[neg_series][max_sentiment:], ignore_index=True)
        print(f'Neg enough: Remaining Count: {remaining["overall"].count()}')

    
    # pos. has not enough
    cur_remaining_idx = 0
    if pos_diff < 0:
        pos_sel = df[pos_series]
        pos_sel = pos_sel.append(remaining[cur_remaining_idx:(-pos_diff)+cur_remaining_idx])
        cur_remaining_idx += -pos_diff
        print(f'Fill positive - Pos Selection: {pos_sel["overall"].count()} - Remainin Idx: {cur_remaining_idx}')
        
    if neu_diff < 0:
        neu_sel = df[neu_series]
        neu_sel = neu_sel.append(remaining[cur_remaining_idx:(-neu_diff)+cur_remaining_idx])
        cur_remaining_idx += -neu_diff
        print(f'Fill Neutral - Neu Selection: {neu_sel["overall"].count()} - Remainin Idx: {cur_remaining_idx}')

        
    if neg_diff < 0:
        neg_sel = df[neg_series]
        neg_sel = neg_sel.append(remaining[cur_remaining_idx:(-neg_diff)+cur_remaining_idx])
        cur_remaining_idx += -neg_diff
        print(f'Fill Negative - Neg Selection: {neg_sel["overall"].count()} - Remainin Idx: {cur_remaining_idx}')

        
    df = pos_sel.append(neu_sel).append(neg_sel)
    return df

In [5]:
root_path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')

aspects = [
    'Apps_for_Android',
    'Baby',
    'Beauty',
    'CDs_and_Vinyl',
    'Cell_Phones_and_Accessories',
    'Clothing_Shoes_and_Jewelry',
    'Digital_Music',
    'Electronics',
    'Grocery_and_Gourmet_Food',
    'Health_and_Personal_Care',
    'Home_and_Kitchen',
    'Kindle_Store',
    'Movies_and_TV',
    'Office_Products',
    'Pet_Supplies',
    'Sports_and_Outdoors',
    'Tools_and_Home_Improvement',
    'Toys_and_Games',
    'Video_Games',
    'Books'
]

In [7]:
df = None

for a in tqdm(aspects):
    fn = f'reviews_{a}_5.json.gz'
    path = os.path.join(root_path, fn)
    print('Parse ' + path)
    
    a_df = getDF(path)
    a_df = process_df(a_df, a)
    a_df.to_pickle(os.path.join(root_path, a + '_processed.pkl'))
    if df is None:
        df = a_df
    else:
        df = df.append(a_df)

A Jupyter Widget

Parse C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\data\data\amazon\reviews_Apps_for_Android_5.json.gz
[524718, 65121, 103098]
Pos enough: Remaining Count: 524718
Neutral enough: Remaining Count: 589839
Neg enough: Remaining Count: 692937
Parse C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\data\data\amazon\reviews_Baby_5.json.gz
[106525, -2745, -2988]
Pos enough: Remaining Count: 106525
Fill Neutral - Neu Selection: 20000 - Remainin Idx: 2745
Fill Negative - Neg Selection: 20000 - Remainin Idx: 5733
Parse C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\data\data\amazon\reviews_Beauty_5.json.gz
[134272, 2248, 1982]
Pos enough: Remaining Count: 134272
Neutral enough: Remaining Count: 136520
Neg enough: Remaining Count: 138502
Parse C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\data\data\amazon\reviews_CDs_and_Vinyl_5.json.gz
[883002, 81824, 72766]
Pos eno

In [8]:
df.to_csv(os.path.join(root_path, 'dataset_processed.csv'))
df.to_pickle(os.path.join(root_path, 'dataset_processed.pkl'))

In [9]:
df.groupby('aspect').count()

,helpful,reviewText,overall,summary,sentiment
aspect,,,,,
Apps_for_Android,60000,60000,60000,60000,60000
Baby,60000,60000,60000,60000,60000
Beauty,60000,60000,60000,60000,60000
Books,60000,60000,60000,60000,60000
CDs_and_Vinyl,60000,60000,60000,60000,60000
Cell_Phones_and_Accessories,60000,60000,60000,60000,60000
Clothing_Shoes_and_Jewelry,60000,60000,60000,60000,60000
Digital_Music,60000,60000,60000,60000,60000
Electronics,60000,60000,60000,60000,60000


In [ ]:
#df.sentiment.describe()

# Generate 80 - 20 - 10 Splits

In [8]:
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')
df = pd.read_pickle(os.path.join(path, 'dataset_processed.pkl'))

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df[['aspect']])
train, val = train_test_split(train, test_size=0.2, random_state=42, stratify=train[['aspect']])



In [11]:
val.count()['overall']/df.count()['overall']

0.18000046930336944

In [16]:
train.groupby('aspect').count()

,helpful,reviewText,overall,summary,sentiment
aspect,,,,,
Apps_for_Android,43200,43200,43200,43200,43200
Baby,43200,43200,43200,43200,43200
Beauty,43200,43200,43200,43200,43200
Books,43200,43200,43200,43200,43200
CDs_and_Vinyl,43200,43200,43200,43200,43200
Cell_Phones_and_Accessories,43200,43200,43200,43200,43200
Clothing_Shoes_and_Jewelry,43200,43200,43200,43200,43200
Digital_Music,43200,43200,43200,43200,43200
Electronics,43200,43200,43200,43200,43200


In [17]:
split_path = os.path.join(path, 'splits')
train.to_pickle(os.path.join(split_path, 'train.pkl'))
test.to_pickle(os.path.join(split_path, 'test.pkl'))
val.to_pickle(os.path.join(split_path, 'val.pkl'))

train.to_csv(os.path.join(split_path, 'train.csv'))
test.to_csv(os.path.join(split_path, 'test.csv'))
val.to_csv(os.path.join(split_path, 'val.csv'))

# Preprocessing & Spell Checking

In [2]:
import hunspell
import re

In [3]:
def en_contraction_removal(text: str) -> str:
    apostrophe_handled = re.sub("’", "'", text)
    # from https://gist.githubusercontent.com/tthustla/74e99a00541264e93c3bee8b2b49e6d8/raw/599100471e8127d6efad446717dc951a10b69777/yatwapart1_01.py
    contraction_mapping = {
                    "i.e.": 'for example',
                    "e.g.": 'for example',
                    "youre": "you are",
                    "youll": "you will",
                    "theyre": "they are", "theyll": "they will",
                    "weve": "we have",
                    "shouldnt": "should not",
                    "dont": "do not",
                    "doesnt": "does not", "doesn": "does not",
                    "didnt": "did not",
                    "wasn": "was not",
                    "arent": "are not", "aren": "are not",
                    "aint": "is not", "isnt": "is not", "isn": "is not",
                    "wouldnt": "would not", "wouldn": "would not",
                    "ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }
    expanded = ' '.join([contraction_mapping[t.lower()] if t.lower() in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    return expanded

In [4]:
hobj = hunspell.HunSpell('/Library/Spelling/en_US.dic', '/Library/Spelling/en_US.aff')


known_words = ['wirelessly', 'hitman', 'Wal-Mart', 'noob', 'subwoofer', 'WTF', 'Waitrose', '<URL>', 'axe', 'TLDR', 'Coca~Cola', 'NPC', 'sci-fi', 'PS3', 'PSX', 'Clooney', 'Schumacher', 'PS2', 'XBOX']


for w in known_words:
    hobj.add(w)

In [5]:
url_regex = r'(?:http(s)?:\/\/)?[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&\(\)\*\+,;=.]+'

def replace_urls_regex(sentence: str, url_token: str = '<URL>') -> str:
    return re.sub(url_regex, url_token, sentence)

def replace_urls(words, url_token: str = '<URL>'):
    return [url_token if (w.lower().startswith('www') or w.lower().startswith('http')) else w for w in words]

def spellcheck_sentence(row) -> str:
    sent = row['reviewText']
    #print(sent)
    to_remove = [',', '(', ')', ':', '?', '&', '/', '*', '!']
    for tr in to_remove:
        sent = sent.replace(tr, ' ')
        
    sent = sent.replace('€™', "'")
    sent = sent.replace('�', "'")
    sent = en_contraction_removal(sent)
    sent = sent.replace("'", ' ')
    sent = replace_urls_regex(sent)


    tokens = sent.split(' ')
    result = []
    for t in tokens:
        if t == ' ':
            continue
        if not hobj.spell(t):
            suggestions = hobj.suggest(t)
            if not suggestions:
                result.append(t)
            else:
                if suggestions[0] == 'e':
                    result.append(t)
                    continue
                result.append(suggestions[0])
                print(f'{t} -> {suggestions[0]}')
        else:
            result.append(t)
    return ' '.join(result)
        
#spellcheck_sentence('This is a tset with a wong wod. Adn now anotheer one why does this notjn workd')

In [ ]:
splits = ['train', 'val', 'test']
path = os.path.join(os.getcwd(), 'data', 'data', 'amazon')


for s in splits:
    print('Split: ' + str(s))
    fn = os.path.join(path, s + '.pkl')
    df = pd.read_pickle(fn)
    df['reviewText'] = df.apply(spellcheck_sentence, axis=1)
    
    fn = os.path.join(path, s + '_sp.pkl')
    df.to_pkl(fn)
    df.to_csv(os.path.join(path, s + '_sp.csv'), sep='|', index=False)

Split: train
"Utterly -> utterly
Monkey" -> Monkey
voice."Monkey" -> monkeyshine
papillons -> pillions
#34;shed -> snowshed
papillon -> papilla
life; -> life
HEA -> HES
wookie -> nookie
feistiness -> festiveness
Stepford -> Step ford
Magda -> Mazda
.BEASTLY -> BEASTLY
Supernail -> Supernal
mani. -> main
le -> Le
socks.I -> socks. I
Wattz -> Watt
nothn -> nothing
bluray -> blurry
"Story -> story
Movie" -> Movie
Brimley -> Brimless
personna -> persona
"Newman -> Newman
"Story -> story
<URL>" -> <URL>
bluray. -> blurry
wordsearch -> word search
washingCon -> washing Con
"Hot" -> Hohhot
"completely -> completely
buds" -> buds
BDP -> BAP
LaRock -> La Rock
KRS-One -> KS-One
KRS-One -> KS-One
"Poetry" -> poetry
"My -> my
Philosophy" -> Philosophy
KRS-One -> KS-One
Run-DMC -> Run-DC
"Some -> some
mc -> MC
talkin -> talking
talkin -> talking
tryin -> trying
walkin -> walking
today." -> today
"Jimmy -> Jimmy
"Illegal -> illegal
reccommend -> recommend
Rundgren -> Reground
genious -> genius
bumb.

"One -> one
Day" -> Day
dismissible -> dismissive
"Money -> money
<URL>" -> <URL>
Geez -> Gee
"Take -> take
Hole" -> Hole
Rami -> Rama
Afuni -> Funafuti
LFMAO -> MAO
"Take -> take
<URL>" -> <URL>
Redfoo -> Red foo
"I -> I
Redfoo -> Red foo
<URL>" -> <URL>
Busta -> Busts
`puts -> puts
"Best -> Best
Night" -> Night
"All -> all
Long" -> Long
"With -> with
You" -> You
Rockingis -> Rocking is
`guilty -> guilty
`Album -> album
`what -> what
"sorry -> sorry
rocking." -> rollicking
olds -> old
MagLite -> Mag Lite
maglight. -> mag light
blu-ray -> blew-ray
blu-ray -> blew-ray
Jarek -> Jared
Lintian -> Quintilian
Zhang -> Hang
Mei -> Me
Mei -> Me
Jarek -> Jared
Jarek -> Jared
Mei -> Me
Jarek -> Jared
Jarek -> Jared
Mei -> Me
Jarek -> Jared
Jarek -> Jared
Jarek -> Jared
Mei -> Me
Meganbook -> Megan book
bad; -> bad
Gamo -> Gamow
PT85 -> PT
dead.I -> dead. I
Dabon -> Damon
Pinaud -> Pinata
bursh -> brush
ThinkThin -> Think Thin
"chocolate -> chocolate
fudge" -> fudge
oversized -> oversize
oversize

dollar". -> dollar
"Mothermania" -> hypothermia
"We -> we
<URL>" -> <URL>
"Ruben -> Ruben
Jets" -> Jets
Mothers" -> Mothers
etc.I -> etc. I
"Jeez -> jeez
looks." -> lookout
"THESE -> THESE
SEEM" -> SEEM
.Up -> . Up
"Uncle -> uncle
Meat". -> Meat
Naturally.I -> Naturally. I
"get -> get
Christmas". -> Christmas
Motels". -> Motels
"you -> you
RYKO -> KORY
ve -> be
Ampex -> Apex
.Uncle -> . Uncle
thats -> thatch
havent -> haven
cd -> CD
materal -> material
thats -> thatch
ok -> OK
"carrying -> carrying
thatThe -> that The
Sharpies.I -> Sharpies. I
touchie -> touche
scrapbooking -> scrap booking
category.I -> category. I
gameplay -> game play
"how -> how
to" -> to
gameplay -> game play
"no -> no
whatsoever." -> whatsoever
"booklet" -> booklet
"behind" -> behind
"the -> the
choose." -> choose
CONTROLSMOUSESelect -> Microelectronics
ButtonAnchor -> Button Anchor
ButtonZoom -> Button Zoom
DownRotate -> Down Rotate
MOuse -> Mouse
RightKEYBOARD1 -> Keyboardist
9CTRL -> CTR
9Spacebar -> backspace

versitallity -> versatility
Evanovich. -> Evangelic
SENDED -> SENSED
REVELON -> REVEL ON
ARDELL--BUT -> AR DELL--BUT
TERRIBLE--DONT -> TERRIBLE--FONT
ok -> OK
charecter -> character
resons -> resins
ok -> OK
charecter -> character
ok -> OK
british -> British
comedities. -> commodities
deflead. -> def lead
"choices" -> choice
cd. -> CD
apos;d -> apposed
apos;m -> apropos
Connor -> Connors
McGuinness -> Guinness
Connor -> Connors
CONOR. -> CONDOR
Mc -> MC
TerraLUX -> Terra Lux
years.I -> years. I
TerraLUX -> Terra Lux
TerraLUX -> Terra Lux
lumens -> lumen
batts -> bats
hours+ -> hours
outputRetains -> output Retains
functionalityEasy -> functionality Easy
installIncreased -> install Increased
lifeDoesn -> lifelessness
heatLED -> heat Led
foreverWhat -> forever What
spendy -> spend
kids; -> kids
Spyro -> Spiro
Gyra -> Gary
Pipo -> Pip
ok. -> OK
ok -> OK
YOu -> You
ADHD -> ADD
Prog -> Prof
placemats -> place mats
#34;real -> arboreal
Alphacolor -> Alpha color
HTC -> HT
"dry" -> dry
9g -> 9

work; -> work
superiors; -> superiors
OCS. -> COS
FFG-12 -> FF-12
JFTR -> RIFT
"I -> I
right." -> right
Caine -> Cain
Maryk -> Mary
order.I -> order. I
Ltjg. -> Lt jg
OOD. -> POD
flatest -> latest
miniscule -> minuscule
quot;To -> quotation
quot; -> quot
quot;Dyer -> quotient
quot;. -> quot
quot;St. -> quote's
quot;. -> quot
"Ryu" -> Ryukyu
-"Animated" -> -animated
"stand" -> handstand
"move -> move
foward" -> forward
"move" -> move
"standing -> standing
around".-"Nina" -> around-"Nina"
"grown-up-nina" -> grown-up-nina"
BOF2-Minigames -> BOFFO-Minigames
-Gameplay -> -Game play
graphics-SNES -> graphics-TINES
alot-May -> a lot-May
"short" -> shortish
standardsBreath -> standards Breath
BOF -> FOB
oldschool -> old school
Layne -> Lane
Staley -> Stanley
ways; -> ways
"Grind" -> grind
Staley -> Stanley
rootsy -> roots
Staley -> Stanley
AIC -> SIC
Staley -> Stanley
"Frogs -> frogs
"Grind" -> grind
"God -> God
"Check -> check
Brain" -> Brain
"Grind" -> grind
"Would -> would
AIC -> SIC
albumD

...

quot;Piano -> quotidian
quot; -> quot
quot;The -> quotient
quot; -> quot
reccomend -> recommend
intrest -> interest
carrear. -> carrier
wheather -> whether
Winona -> Wino
Jolie -> Josie
oscar -> Oscar
exactely -> exactly
Reccomended. -> Recommended
"going -> going
green." -> green
mids -> dims
Dre -> Dee
Suge -> Sue
Dre -> Dee
2Pac -> PAC
Doggfather -> Godfather
Dre -> Dee
Dre -> Dee
midas -> Midas
Dre -> Dee
"Been -> been
Dre -> Dee
"I -> I
ve -> be
<URL>" -> <URL>
Dre -> Dee
KRS-One -> KS-One
Nas -> Na
"East -> East
Killas -> Villas
Dre-produced -> Dee-produced
"Natural -> natural
Killas" -> Skill's
Dre -> Dee
bad@ss -> badness
Dre -> Dee
"Sh -> sh
non-descript -> non-descriptor
Dre -> Dee
"I -> I
reppin -> rep pin
R+B -> RIB
"Got -> got
Open" -> Open
Dre -> Dee
"No -> No
Diggity." -> Dignity
Dre -> Dee
da -> DA
Wilcher. -> Witchery
Nowl -> Noel
Sharief -> Sharif
Dre -> Dee
half-@ssed -> half-dissed
srcub -> scrub
"feel" -> feel
"chemical" -> chemical
"chemical" -> chemical
"scent" -

bipod -> biped
bipod. -> biped
bipod -> biped
Fellowes -> Fellows
covers; -> covers
binders; -> binders
Fellowes -> Fellows
shoulder2 -> shoulder
unsliced -> unlicensed
fennel2 -> fennel
corriander2 -> coriander
ginger1 -> ginger
mustard1 -> mustard
powder2 -> powder
pepper1 -> pepper
watersprinkling -> water sprinkling
saltUse -> salt Use
"The -> the
mishmas -> mishmash
Ghostbusters -> Ghost busters
"special" -> especial
"must -> must
have" -> have
rosehip -> rose hip
grapeseed -> rapeseed
reflux -> reflex
#34;just -> justness
#34;right -> rightist
38528-033Hoover -> 38528-Hoover
OEM -> UM
june -> June
$<URL> -> <URL>
havent -> haven
ok.. -> OK
spackle -> Spackle
surface."Regular" -> regularization
spackle -> Spackle
spackle -> Spackle
RESULTS->Fresh -> RESULTS-fresh
spackle -> Spackle
spackle -> Spackle
-->Use -> --use
4oz -> oz
<URL>>Do -> <URL>
spackle -> Spackle
minutes-->Do -> minutes--do
<URL>>Be -> <URL>
<URL>>If -> <URL>
well.->Use -> well.-use
1" -> 1
2" -> 2
2" -> 2
3" -> 3


quot;. -> quot
favourite -> favorite
quot; -> quot
booket -> booklet
3x -> 3
jobs.I -> jobs. I
Dostoyevsky -> Dostoevsky
Sozhenitsyn -> Solzhenitsyn
"One -> one
Denisovich". -> Hoydenish
fictional; -> fictional
not.I -> not. I
Creedence -> Credence
Clearwater -> Clear water
"Proud -> proud
Mary" -> Mary
`n -> n
"Born -> Born
"Keep -> keep
Chooglin -> Schooling
"Graveyard -> graveyard
Train" -> Train
"Good -> Good
Fogerty -> Forgery
"Green -> Green
River" -> Rivers
"Bad -> bad
"Penthouse -> penthouse
Pauper" -> Pauper
"Bootleg" -> bootleg
CCR -> CC
Otterbox -> Otter box
S3 -> S
Otterbox -> Otter box
Otterbox -> Otter box
"protector" -> protectorate
Otterbox. -> Otter box
"power" -> power
Dj -> DJ
cd -> CD
"Love -> Love
Hangover" -> Hangover
Dinan -> Diann
throww -> throw
squirells -> squirrels
itll -> till
`plots -> plots
"Thoughtful -> thoughtful
Neighbor" -> Neighbor
"Never -> never
s" -> s
"Dumb -> dumb
Criminals" -> Criminals
"lost" -> lost
"same -> same
old" -> old
"must -> must
ow

sideburn -> sideburns
Oster -> Ester
haircutting. -> hair cutting
"My -> my
Touch" -> Touch
4G -> 4
Hulu -> Hutu
chiseldrivers -> chisel drivers
convenient.I -> convenient. I
Felo -> Flo
Ergonics -> Ergonomics
Wera -> Ware
Chiseldrivers -> Chisel drivers
Felo -> Flo
Felo -> Flo
Wera -> Ware
do.I -> do. I
OXO -> OX
$5 -> 5
red; -> red
beam; -> beam
beam; -> beam
combined; -> combined
$5 -> 5
Petzl -> Petal
Tikkina -> Katina
lemon; -> lemon
PFG -> PF
unpowered -> powered
FG -> FF
PFG -> PF
PFG -> PF
FG -> FF
bulbousness -> nebulousness
PFG -> PF
FG -> FF
PFG -> PF
FG. -> FF
PFG -> PF
FG -> FF
PFG -> PF
.Third -> . Third
PFG -> PF
FG -> FF
PFG -> PF
PFG -> PF
FG -> FF
PFG -> PF
blade; -> blade
PFG. -> PF
"farther" -> farther
removerAdding -> remover Adding
"Jesus -> Jesus
gangster"Take -> gangbusters
amongst -> among st
"Little -> Little
Italy" -> Italy
"I -> I
you" -> you
"He -> He
am" -> am
moder -> mode
criticise -> criticism
de -> DE
tinyCoaster -> tiny Coaster
Finishhere -> Finish he

repositioned -> re positioned
cube.I -> cube. I
"cube -> cube
you" -> you
Amazon.I -> Amazon. I
nicer.I -> nicer. I
"books" -> bookshop
cardstock -> card stock
.I -> I
trailor -> tailor
quot;SCREAM -> buttercream
quot; -> quot
quot;I -> quotient
quot;... -> quot
evo. -> eve
VideoSecu -> Videos ecu
catalogue -> uncatalogued
Peart -> Petra
Rutsey. -> Rukeyser
Peart -> Petra
BUt -> But
albuns -> albums
2Definitely -> 2 Definitely
"<URL> -> <URL>
<URL>" -> <URL>
cups". -> cups
velcro -> Velcro
e.g -> erg
Molle -> Moll
neo -> Neo
Fleigerduff -> Fleischer
shutting; -> shutting
Molle -> Moll
Molles. -> Moll's
Doppelduffel -> Doppelganger
Molle -> Moll
bag; -> bag
FleigerDuff -> Fleischer
middle; -> middle
useable -> usable
pull; -> pull
neoprened -> neoprene
DoppelDuff. -> Copperfield
DopDuff -> Duffy
encrouch -> encroach
Molle -> Moll
Molle -> Moll
narrow; -> narrow
eg -> eh
Maxped -> Maxed
Youtube -> YouTube
vids -> bids
Maxped -> Maxed
e.g -> erg
.The -> . The
attached; -> attached
YKK -> 